In [281]:
# setup

import pandas as pd
import numpy as np
import altair as alt
import warnings
warnings.filterwarnings('ignore')


In [282]:
pip install vega_datasets


python(55498) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Note: you may need to restart the kernel to use updated packages.


In [1]:
# combine all regional datasets

regions = {
    "Africa": "Africa_aggregated_data_up_to-2025-10-18.csv",
    "Asia-Pacific": "Asia-Pacific_aggregated_data_up_to-2025-10-18.csv",
    "Europe-Central-Asia": "Europe-Central-Asia_aggregated_data_up_to-2025-10-18.csv",
    "Latin-America-the-Caribbean": "Latin-America-the-Caribbean_aggregated_data_up_to-2025-10-18.csv",
    "Middle-East": "Middle-East_aggregated_data_up_to-2025-10-18.csv",
    "US-and-Canada": "US-and-Canada_aggregated_data_up_to-2025-10-18.csv"
}

# Load all datasets
dfs = {}
for region_name, file_path in regions.items():
    dfs[region_name] = pd.read_csv(file_path)
    print(f"Loaded {region_name}: {len(dfs[region_name])} rows")

# Combine into single dataframe
df = pd.concat(dfs.values(), ignore_index=True)
print(f"Combined dataset: {len(df)} total rows")

NameError: name 'pd' is not defined

**Data Preprocessing**

In [284]:
# Convert WEEK to datetime
df['WEEK'] = pd.to_datetime(df['WEEK'])
df['YEAR'] = df['WEEK'].dt.year
df['MONTH'] = df['WEEK'].dt.month
df['YEAR_MONTH'] = df['WEEK'].dt.to_period('M')

# Ensure numeric columns
df['EVENTS'] = pd.to_numeric(df['EVENTS'], errors='coerce').fillna(0)
df['FATALITIES'] = pd.to_numeric(df['FATALITIES'], errors='coerce').fillna(0)
df['POPULATION_EXPOSURE'] = pd.to_numeric(df['POPULATION_EXPOSURE'], errors='coerce').fillna(0)

df

WEEK           REGION        COUNTRY   ADMIN1  \
0      2004-10-23  Northern Africa        Algeria    Adrar   
1      2005-04-23  Northern Africa        Algeria    Adrar   
2      2005-06-25  Northern Africa        Algeria    Adrar   
3      2008-12-13  Northern Africa        Algeria    Adrar   
4      2009-04-18  Northern Africa        Algeria    Adrar   
...           ...              ...            ...      ...   
884211 2025-01-25    North America  United States  Wyoming   
884212 2025-04-05    North America  United States  Wyoming   
884213 2025-04-19    North America  United States  Wyoming   
884214 2025-05-03    North America  United States  Wyoming   
884215 2025-10-04    North America  United States  Wyoming   

                    EVENT_TYPE SUB_EVENT_TYPE  EVENTS  FATALITIES  \
0                      Battles    Armed clash       1           2   
1                      Battles    Armed clash       1           0   
2                      Battles    Armed clash       1          14   
3                      Battles    Armed clash       1           3   
4                      Battles    Armed clash       1           2   
...                        ...            ...     ...         ...   
884211  Strategic developments          Other       1           0   
884212  Strategic developments          Other       1           0   
884213  Strategic developments          Other       1           0   
884214  Strategic developments          Other       1           0   
884215  Strategic developments          Other       1           0   

        POPULATION_EXPOSURE           DISORDER_TYPE      ID  \
0                       0.0      Political violence    47.0   
1                       0.0      Political violence    47.0   
2                       0.0      Political violence    47.0   
3                       0.0      Political violence    47.0   
4                       0.0      Political violence    47.0   
...                     ...                     ...     ...   
884211                  0.0  Strategic developments  3533.0   
884212                  0.0  Strategic developments  3533.0   
884213                  0.0  Strategic developments  3533.0   
884214                  0.0  Strategic developments  3533.0   
884215                  0.0  Strategic developments  3533.0   

        CENTROID_LATITUDE  CENTROID_LONGITUDE  YEAR  MONTH YEAR_MONTH  
0                 26.4839             -1.3880  2004     10    2004-10  
1                 26.4839             -1.3880  2005      4    2005-04  
2                 26.4839             -1.3880  2005      6    2005-06  
3                 26.4839             -1.3880  2008     12    2008-12  
4                 26.4839             -1.3880  2009      4    2009-04  
...                   ...                 ...   ...    ...        ...  
884211            42.9879           -107.5485  2025      1    2025-01  
884212            42.9879           -107.5485  2025      4    2025-04  
884213            42.9879           -107.5485  2025      4    2025-04  
884214            42.9879           -107.5485  2025      5    2025-05  
884215            42.9879           -107.5485  2025     10    2025-10  

[884216 rows x 16 columns]

In [285]:
# Summary Stats


total_events = df['EVENTS'].sum()
total_fatalities = df['FATALITIES'].sum()
unique_countries = df['COUNTRY'].nunique()
unique_regions = df['REGION'].nunique()
time_range = f"{df['WEEK'].min().date()} to {df['WEEK'].max().date()}"
unique_event_types = df['EVENT_TYPE'].nunique()

print(f"Total Events: {total_events:,.0f}")
print(f"Total Fatalities: {total_fatalities:,.0f}")
print(f"Unique Countries: {unique_countries}")
print(f"Unique Regions: {unique_regions}")
print(f"Time Range: {time_range}")
print(f"Unique Event Types: {unique_event_types}")

Total Events: 2,772,023
Total Fatalities: 2,399,712
Unique Countries: 238
Unique Regions: 17
Time Range: 1996-12-28 to 2025-10-18
Unique Event Types: 6


In [286]:
# summary stats table for regions
summary_by_region = df.groupby('REGION').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum',
    'COUNTRY': 'nunique',
    'EVENT_TYPE': 'nunique'
}).round(0).reset_index()
summary_by_region.columns = ['Region', 'Total Events', 'Total Fatalities', 'Countries', 'Event Types']
summary_by_region = summary_by_region.sort_values('Total Events', ascending=False)

display(summary_by_region)

Region  Total Events  Total Fatalities  Countries  \
8                 Middle East        555682            513190         15   
6                      Europe        529764            225144         53   
13                 South Asia        370430             75712          7   
12              South America        238377             93066         14   
9               North America        212065             66217          5   
14             Southeast Asia        156422            122488         11   
5              Eastern Africa        141703            351946         19   
2   Caucasus and Central Asia        109242            212080          9   
10            Northern Africa        101877            195428          6   
16             Western Africa        100437            220716         16   
4                   East Asia         82592               248          6   
7               Middle Africa         72132            284241          9   
15            Southern Africa         34681              5170          8   
3             Central America         34674             15055          7   
1                   Caribbean         24490             17666         28   
11                    Oceania          7446              1345         24   
0                  Antarctica             9                 0          1   

    Event Types  
8             6  
6             6  
13            6  
12            6  
9             6  
14            6  
5             6  
2             6  
10            6  
16            6  
4             6  
7             6  
15            6  
3             6  
1             6  
11            6  
0             1

**Data aggr for visualzations**

In [287]:
# Top 15 countries by events
top_countries = df.groupby('COUNTRY').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index().sort_values('EVENTS', ascending=False).head(15)

# Monthly trend
monthly_trend = df.groupby(df['WEEK'].dt.to_period('M')).agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()
monthly_trend['WEEK'] = monthly_trend['WEEK'].dt.to_timestamp()

# Yearly trend
yearly_trend = df.groupby('YEAR').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

# Top event types
top_event_types = df.groupby('EVENT_TYPE')['EVENTS'].sum().reset_index().sort_values('EVENTS', ascending=False)

# Country-level aggregation for choropleth
country_stats = df.groupby('COUNTRY').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum',
    'POPULATION_EXPOSURE': 'sum'
}).reset_index()
country_stats['FATALITY_RATE'] = country_stats['FATALITIES'] / (country_stats['EVENTS'] + 1)

# Geographic scatter (by country centroid)
map_data = df.groupby(['COUNTRY', 'CENTROID_LATITUDE', 'CENTROID_LONGITUDE']).agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"15 Top Countries")
display(top_countries)
print(f"Monthly Trends")
display(monthly_trend)
print(f"Yearly Trends")
display(yearly_trend)
print(f"Events by region and month")
display(region_month)
print(f"Top event types")
display(top_event_types)
print(f"Country-level aggregation for choropleth")
display(country_stats)
print(f"Geographic scatter (by country centroid)")
display(map_data)


15 Top Countries


COUNTRY  EVENTS  FATALITIES
222        Ukraine  269387      217249
97           India  195161       13359
207          Syria  145292      141446
135         Mexico  111153       65708
160       Pakistan  108435       53281
234          Yemen  107829      165446
31          Brazil  102820       54864
144        Myanmar   96504       98567
225  United States   91948         496
162      Palestine   82404       71263
0      Afghanistan   68519      203244
100           Iraq   66004      107355
197        Somalia   52623       89449
217         Turkey   46596        9700
175         Russia   46133        7220

Monthly Trends


WEEK  EVENTS  FATALITIES
0   1996-12-01      27          72
1   1997-01-01     214        3369
2   1997-02-01     143        1753
3   1997-03-01     265        1612
4   1997-04-01     222        2017
..         ...     ...         ...
342 2025-06-01   32602       18892
343 2025-07-01   30655       17295
344 2025-08-01   37324       19951
345 2025-09-01   33175       15809
346 2025-10-01   22626        9874

[347 rows x 3 columns]

Yearly Trends


...

EVENT_TYPE   EVENTS
2                    Protests  1130708
1  Explosions/Remote violence   500835
0                     Battles   460185
5  Violence against civilians   310934
4      Strategic developments   191609
3                       Riots   177752

Country-level aggregation for choropleth


COUNTRY  EVENTS  FATALITIES  POPULATION_EXPOSURE  \
0              Afghanistan   68519      203244          909685303.0   
1    Akrotiri and Dhekelia      15           0               4469.0   
2                  Albania    1384           4           24590278.0   
3                  Algeria   12799       13801           65263229.0   
4           American Samoa       7           0              12324.0   
..                     ...     ...         ...                  ...   
233      Wallis and Futuna      27           0               7560.0   
234                  Yemen  107829      165446          231902879.0   
235                 Zambia    1578         355           11602267.0   
236               Zimbabwe    6733         635           12117596.0   
237               eSwatini     842         148            3638987.0   

     FATALITY_RATE  
0         2.966200  
1         0.000000  
2         0.002888  
3         1.078203  
4         0.000000  
..             ...  
233       0.000000  
234       1.534323  
235       0.224826  
236       0.094298  
237       0.175563  

[238 rows x 5 columns]

Geographic scatter (by country centroid)


COUNTRY  CENTROID_LATITUDE  CENTROID_LONGITUDE  EVENTS  FATALITIES
0     Afghanistan            30.8409             62.4753     697        1379
1     Afghanistan            31.0757             65.6926    4948       18939
2     Afghanistan            31.3969             64.0428    6243       17785
3     Afghanistan            32.2774             67.0193    2211        6636
4     Afghanistan            32.4966             62.4100    1988        7441
...           ...                ...                 ...     ...         ...
3526     Zimbabwe           -16.6813             31.6098     554          79
3527     eSwatini           -27.0244             31.3137     104          27
3528     eSwatini           -26.5546             31.8619     122          22
3529     eSwatini           -26.5264             31.1888     306          65
3530     eSwatini           -26.1033             31.3640     310          34

[3531 rows x 5 columns]

In [288]:
alt.data_transformers.disable_max_rows()

# Geojson
# import requests

WORLD_GEOJSON_URL = "https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson"

geo = requests.get(WORLD_GEOJSON_URL).json()
geo_countries = [f['properties']['name'] for f in geo['features']]

# Fix country name mismatches
fix_names = {
    'Democratic Republic of Congo': 'Democratic Republic of the Congo',
    'Republic of Congo': 'Republic of the Congo',
    'Tanzania': 'United Republic of Tanzania',
    'eSwatini': 'Swaziland',
    'United States': 'United States of America',
    'Palestine': 'Palestine, State of',
    'Vatican City': 'Holy See',
    'Serbia': 'Republic of Serbia',
    'Cape Verde': 'Cabo Verde',
    'North Macedonia': 'Macedonia',
    'Iran': 'Iran (Islamic Republic of)',
    'Syria': 'Syrian Arab Republic',
    'Russia': 'Russian Federation'
}

country_stats['COUNTRY'] = country_stats['COUNTRY'].replace(fix_names)

# Remove Bermuda (covers entire map, skews visualization)
country_stats = country_stats[country_stats['COUNTRY'] != 'Bermuda']
print("Removed Bermuda from dataset (causes map distortion).")

# Keep only countries found in GeoJSON
country_stats = country_stats[country_stats['COUNTRY'].isin(geo_countries)].reset_index(drop=True)
print(f"Matched {len(country_stats)} countries to GeoJSON features.")




Removed Bermuda from dataset (causes map distortion).
Matched 210 countries to GeoJSON features.


**VISUALIZATIONS**

In [289]:
# Top 15 countries by events bar graph
chart1 = alt.Chart(top_countries.reset_index(drop=True)).mark_bar().encode(
    y=alt.Y('COUNTRY:N', sort='-x', title='Country'),
    x=alt.X('EVENTS:Q', title='Total Events'),
    color=alt.Color('FATALITIES:Q', scale=alt.Scale(scheme='reds'), title='Fatalities'),
    tooltip=['COUNTRY', 'EVENTS:Q', 'FATALITIES:Q']
).properties(
    width=700, height=400,
    title='Top 15 Countries by Total Conflict Events'
)

chart1

alt.Chart(...)

In [290]:
# Monthy trends with fatalites

chart2_events = alt.Chart(monthly_trend).mark_line(point=True, color='steelblue', size=2).encode(
    x=alt.X('WEEK:T', title='Date'),
    y=alt.Y('EVENTS:Q', title='Events', axis=alt.Axis(labelColor='steelblue', titleColor='steelblue')),
    tooltip=['WEEK:T', 'EVENTS:Q', 'FATALITIES:Q']
).properties(
    width=900, height=350,
    title='Monthly Trend: Conflict Events Over Time'
).interactive()

chart2_fatalities = alt.Chart(monthly_trend).mark_line(point=True, color='darkred', size=2).encode(
    x=alt.X('WEEK:T', title='Date'),
    y=alt.Y('FATALITIES:Q', title='Fatalities', axis=alt.Axis(orient='right', labelColor='darkred', titleColor='darkred')),
    tooltip=['WEEK:T', 'EVENTS:Q', 'FATALITIES:Q']
)

monthly_combined = alt.layer(chart2_events, chart2_fatalities).resolve_scale(y='independent')

monthly_combined

alt.LayerChart(...)

In [302]:
# Yearly Trends

chart3 = alt.Chart(yearly_trend).mark_bar(color='orange').encode(
    x=alt.X('YEAR:O', title='Year'),
    y=alt.Y('EVENTS:Q', title='Total Events'),
    color=alt.Color('FATALITIES:Q', scale=alt.Scale(scheme='reds'), title='Fatalities'),
    tooltip=['YEAR', 'EVENTS:Q', 'FATALITIES:Q']
).properties(
    width=600, height=300,
    title='Annual Conflict Events: Yearly Aggregation'
)

chart3

alt.Chart(...)

In [292]:
# 4. REGIONAL CONCENTRATION

regional_events = df.groupby('REGION')['EVENTS'].sum().reset_index().sort_values('EVENTS', ascending=False)
chart4 = alt.Chart(regional_events).mark_bar().encode(
    y=alt.Y('REGION:N', sort='-x', title='Region'),
    x=alt.X('EVENTS:Q', title='Total Events'),
    color=alt.value('teal'),
    tooltip=['REGION', 'EVENTS:Q']
).properties(
    width=600, height=250,
    title='Regional Concentration: Total Events by Region'
)

chart4

alt.Chart(...)

In [293]:
# 5. EVENT TYPE DISTRIBUTION

chart5 = alt.Chart(top_event_types.head(10)).mark_bar(color='teal').encode(
    y=alt.Y('EVENT_TYPE:N', sort='-x', title='Event Type'),
    x=alt.X('EVENTS:Q', title='Total Events'),
    tooltip=['EVENT_TYPE', 'EVENTS:Q']
).properties(
    width=700, height=300,
    title='Top 10 Event Types by Frequency'
)

chart5

alt.Chart(...)

In [294]:
country_stats

COUNTRY  EVENTS  FATALITIES  POPULATION_EXPOSURE  FATALITY_RATE
0          Afghanistan   68519      203244          909685303.0       2.966200
1              Albania    1384           4           24590278.0       0.002888
2              Algeria   12799       13801           65263229.0       1.078203
3       American Samoa       7           0              12324.0       0.000000
4              Andorra      30           0             276206.0       0.000000
..                 ...     ...         ...                  ...            ...
205            Vietnam     569         147            1815293.0       0.257895
206  Wallis and Futuna      27           0               7560.0       0.000000
207              Yemen  107829      165446          231902879.0       1.534323
208             Zambia    1578         355           11602267.0       0.224826
209           Zimbabwe    6733         635           12117596.0       0.094298

[210 rows x 5 columns]

In [295]:
# ============================================================================
# CHART 8-10: INTERACTIVE CHOROPLETH MAPS WITH DROPDOWN
# ============================================================================

WORLD_GEOJSON_URL = "https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json"

# Define metrics to display
metrics = ['EVENTS', 'FATALITIES', 'FATALITY_RATE']

# Create dropdown selector
metric_sel = alt.selection_point(
    fields=['Metric'],
    bind=alt.binding_select(options=metrics, name='View Metric: '),
    value='EVENTS'
)

# Background (base countries layer in light gray)
base = alt.Chart(
    alt.Data(url=WORLD_GEOJSON_URL, format={'type': 'json', 'property': 'features'})
).mark_geoshape(stroke='white', strokeWidth=0.25).encode(
    color=alt.value('#eaeaea')
).project('mercator')

# Interactive choropleth layer
choropleth = alt.Chart(
    alt.Data(url=WORLD_GEOJSON_URL, format={'type': 'json', 'property': 'features'})
).mark_geoshape(stroke='white', strokeWidth=0.25).transform_lookup(
    lookup='properties.name',
    from_=alt.LookupData(data=country_stats, key='COUNTRY', fields=['EVENTS', 'FATALITIES', 'FATALITY_RATE'])
).transform_fold(
    metrics,
    as_=['Metric', 'Value']
).transform_filter(
    metric_sel
).encode(
    color=alt.Color(
        'Value:Q',
        scale=alt.Scale(scheme='reds'),
        title='Value'
    ),
    tooltip=[
        alt.Tooltip('properties.name:N', title='Country'),
        alt.Tooltip('Metric:N', title='Metric'),
        alt.Tooltip('Value:Q', title='Value', format='.2f')
    ]
).project(
    type='mercator'
).add_params(
    metric_sel
)

In [300]:
# Combine base + interactive layer
chart8 = (base + choropleth).properties(
    width=1200, height=700,
    title='Interactive Choropleth: Switch between EVENTS / FATALITIES / FATALITY_RATE'
)

chart8

alt.LayerChart(...)

In [301]:
# For individual fixed maps, keep chart9 and chart10 as static versions
chart9 = alt.Chart(
    alt.Data(url=WORLD_GEOJSON_URL, format={'type': 'json', 'property': 'features'})
).mark_geoshape(stroke='white', strokeWidth=0.25).transform_lookup(
    lookup='properties.name',
    from_=alt.LookupData(data=country_stats, key='COUNTRY', fields=['EVENTS', 'FATALITIES'])
).encode(
    color=alt.Color('FATALITIES:Q', scale=alt.Scale(scheme='purples'), title='Total Fatalities'),
    tooltip=[
        alt.Tooltip('properties.name:N', title='Country'),
        alt.Tooltip('FATALITIES:Q', title='Fatalities', format=','),
        alt.Tooltip('EVENTS:Q', title='Events', format=',')
    ]
).project(
    type='mercator'
).properties(
    width=1200, height=700,
    title='Map 3: Fatalities by Country (Static View)'
)

chart9

alt.Chart(...)

In [298]:
# CHART 11: FATALITY INTENSITY BY COUNTRY (BAR)
# ============================================================================

fatality_countries = df.groupby('COUNTRY').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()
fatality_countries['INTENSITY'] = fatality_countries['FATALITIES'] / (fatality_countries['EVENTS'] + 1)
fatality_countries = fatality_countries.sort_values('FATALITIES', ascending=False).head(12)

chart11 = alt.Chart(fatality_countries.reset_index(drop=True)).mark_bar().encode(
    y=alt.Y('COUNTRY:N', sort='-x', title='Country'),
    x=alt.X('FATALITIES:Q', title='Total Fatalities'),
    color=alt.Color('INTENSITY:Q', scale=alt.Scale(scheme='reds'), title='Fatality Intensity'),
    tooltip=['COUNTRY', 'FATALITIES:Q', 'EVENTS:Q', 'INTENSITY:Q']
).properties(
    width=700, height=350,
    title='Top 12 Countries by Total Fatalities (Color: Fatality Intensity)'
)

chart11

alt.Chart(...)

In [299]:
top_country = top_countries.iloc[0]
top_region = summary_by_region.iloc[0]

print(f"""
1. TEMPORAL PATTERNS:
   - The dataset covers {time_range}, capturing {df['YEAR'].nunique()} years of conflict data.
   - Monthly trends reveal {'increasing' if monthly_trend['EVENTS'].iloc[-1] > monthly_trend['EVENTS'].iloc[0] else 'decreasing'} 
     patterns in recent months, with total events ranging from 
     {monthly_trend['EVENTS'].min():,.0f} to {monthly_trend['EVENTS'].max():,.0f} per month.

2. SPATIAL PATTERNS:
   - Conflict activity is concentrated in {unique_countries} countries across {unique_regions} regions.
   - {top_country['COUNTRY']} leads with {top_country['EVENTS']:,.0f} events and {top_country['FATALITIES']:,.0f} fatalities.
   - {top_region['Region']} accounts for the most events ({top_region['Total Events']:,.0f}), 
     indicating regional concentration of political violence.

3. EVENT CHARACTERISTICS:
   - Most common event type: {top_event_types.iloc[0]['EVENT_TYPE']}
   - Fatality intensity varies significantly: some countries experience 
     high-intensity events while others have frequent but lower-casualty incidents.

4. GEOGRAPHIC CONCENTRATION:
   - Events cluster in specific geographic zones (visible in geographic scatter map and choropleth maps).
   - Multiple map types reveal different dimensions:
     * Scatter map: Shows event density and fatality magnitude by location
     * Events choropleth: Identifies countries with highest absolute conflict counts
     * Fatalities choropleth: Highlights deadliest regions
     * Intensity choropleth: Reveals where events are most severe (fatalities per event)

5. PREPARATION FOR GOOGLE TRENDS INTEGRATION:
   - This descriptive analysis provides the baseline understanding of conflict patterns.
   - Next steps: Identify top countries/regions for Google Trends correlation analysis.
   - Consider time-lagging and seasonality when matching trends data.
   - Geographic maps will help align search behavior with conflict locations.
""")


1. TEMPORAL PATTERNS:
   - The dataset covers 1996-12-28 to 2025-10-18, capturing 30 years of conflict data.
   - Monthly trends reveal increasing 
     patterns in recent months, with total events ranging from 
     27 to 40,750 per month.

2. SPATIAL PATTERNS:
   - Conflict activity is concentrated in 238 countries across 17 regions.
   - Ukraine leads with 269,387 events and 217,249 fatalities.
   - Middle East accounts for the most events (555,682), 
     indicating regional concentration of political violence.

3. EVENT CHARACTERISTICS:
   - Most common event type: Protests
   - Fatality intensity varies significantly: some countries experience 
     high-intensity events while others have frequent but lower-casualty incidents.

4. GEOGRAPHIC CONCENTRATION:
   - Events cluster in specific geographic zones (visible in geographic scatter map and choropleth maps).
   - Multiple map types reveal different dimensions:
     * Scatter map: Shows event density and fatality magnitude by l